### Test of data instantiation

In [ ]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader



In [ ]:

class TestDataLoader(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.as_tensor(images)
        self.labels = torch.as_tensor(labels)
        self.transform = transform
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label
        
img = torch.ones(6, 3)
labels = [0, 1, 1, 1, 0, 0]

loader = TestDataLoader(img, labels)
print(loader[4])



(tensor([1., 1., 1.]), tensor([0]))
